# 1. QONNX - How to work with models

We first start to create and manipulate an ONNX model and learn to use some QONNX functions to work with it.

NOTE: most of this is based on https://github.com/onnx/tutorials and https://github.com/fastmachinelearning/qonnx/tree/main/notebooks

In [ ]:
# lets first install qonnx and onnx
%pip install qonnx
%pip install onnx

### 1.1. How to create a simple ONNX model

To explain how to create an ONNX model a simple example with mathematical operations is used. All nodes are from the [standard operations library of ONNX](https://github.com/onnx/onnx/blob/master/docs/Operators.md).

First ONNX is imported, then the helper function can be used to make a node.

In [ ]:
import onnx
from qonnx.util.basic import qonnx_make_model

Add1_node = onnx.helper.make_node(
    'Add',
    inputs=['in1', 'in2'],
    outputs=['sum1'],
    name='Add1'
)

The first attribute of the node is the operation type. In this case it is `'Add'`, so it is an adder node. Then the input names are passed to the node and at the end a name is assigned to the output.
    
For this example we want two other adder nodes, one abs node and the output shall be rounded so one round node is needed.

In [ ]:
Add2_node = onnx.helper.make_node(
    'Add',
    inputs=['sum1', 'in3'],
    outputs=['sum2'],
    name='Add2',
)

Abs_node = onnx.helper.make_node(
    'Abs',
    inputs=['sum2'],
    outputs=['abs1'],
    name='Abs'
)

Add3_node = onnx.helper.make_node(
    'Add',
    inputs=['abs1', 'abs1'],
    outputs=['sum3'],
    name='Add3',
)

Round_node = onnx.helper.make_node(
    'Round',
    inputs=['sum3'],
    outputs=['out1'],
    name='Round',
)

The names of the inputs and outputs of the nodes give already an idea of the structure of the resulting graph. In order to integrate the nodes into a graph environment, the inputs and outputs of the graph have to be specified first. In ONNX all data edges are processed as tensors. So with onnx helper function tensors value infos are created for the input and output tensors of the graph. Float from ONNX is used as data type.

In [ ]:
in1 = onnx.helper.make_tensor_value_info("in1", onnx.TensorProto.FLOAT, [4, 4])
in2 = onnx.helper.make_tensor_value_info("in2", onnx.TensorProto.FLOAT, [4, 4])
in3 = onnx.helper.make_tensor_value_info("in3", onnx.TensorProto.FLOAT, [4, 4])
out1 = onnx.helper.make_tensor_value_info("out1", onnx.TensorProto.FLOAT, [4, 4])

Now the graph can be built. First all nodes are passed. Here it is to be noted that it requires a certain sequence. The nodes must be instantiated in their dependencies to each other. This means Add2 must not be listed before Add1, because Add2 depends on the result of Add1. A name is then assigned to the graph. This is followed by the inputs and outputs.

`value_info` of the graph contains the remaining tensors within the graph. When creating the nodes we have already defined names for the inner data edges and now these are assigned tensors of the datatype float and a certain shape.

In [ ]:
graph = onnx.helper.make_graph(
      nodes=[
          Add1_node,
          Add2_node,
          Abs_node,
          Add3_node,
          Round_node,
      ],
      name="simple_graph",
      inputs=[in1, in2, in3],
      outputs=[out1],
      value_info=[
          onnx.helper.make_tensor_value_info("sum1", onnx.TensorProto.FLOAT, [4, 4]),
          onnx.helper.make_tensor_value_info("sum2", onnx.TensorProto.FLOAT, [4, 4]),
          onnx.helper.make_tensor_value_info("abs1", onnx.TensorProto.FLOAT, [4, 4]),
          onnx.helper.make_tensor_value_info("sum3", onnx.TensorProto.FLOAT, [4, 4]),
      ],
  )

**Important**: In our example, the shape of the tensors does not change during the calculation. This is not always the case. So you have to make sure that you specify the shape correctly.

Now a model can be created from the graph and saved using the `.save` function. The model is saved in .onnx format and can be reloaded with `onnx.load()`. This also means that you can easily share your own model in .onnx format with others.

In [ ]:
onnx_model = qonnx_make_model(graph, producer_name="simple-model")
onnx.save(onnx_model, 'simple_model.onnx')

To visualize the created model, [Netron](https://github.com/lutzroeder/netron) can be used. Netron is a visualizer for neural network, deep learning and machine learning models. We provide a utility function for visualization with Netron, which we import and use in the following.

In [ ]:
%pip install netron
import netron
import os
from IPython.display import IFrame

def showInNetron(model_filename: str, localhost_url: str = None, port: int = None):
    """Shows a ONNX model file in the Jupyter Notebook using Netron.

    :param model_filename: The path to the ONNX model file.
    :type model_filename: str

    :param localhost_url: The IP address used by the Jupyter IFrame to show the model.
     Defaults to localhost.
    :type localhost_url: str, optional

    :param port: The port number used by Netron and the Jupyter IFrame to show
     the ONNX model.  Defaults to 8081.
    :type port: int, optional

    :return: The IFrame displaying the ONNX model.
    :rtype: IPython.lib.display.IFrame
    """
    try:
        port = port or int(os.getenv("NETRON_PORT", default="8081"))
    except ValueError:
        port = 8081
    localhost_url = localhost_url or os.getenv("LOCALHOST_URL", default="localhost")
    netron.start(model_filename, address=("0.0.0.0", port), browse=False)
    return IFrame(src=f"http://{localhost_url}:{port}/", width="100%", height=400)

In [ ]:
showInNetron('simple_model.onnx')

Netron also allows you to interactively explore the model. If you click on a node, the node attributes will be displayed. 

In order to test the resulting model, a function is first written in Python that calculates the expected output. Because numpy arrays are to be used, numpy is imported first.

In [ ]:
%pip install numpy
import numpy as np

def expected_output(in1, in2, in3):
    sum1 = np.add(in1, in2)
    sum2 = np.add(sum1, in3)
    abs1 = np.absolute(sum2)
    sum3 = np.add(abs1, abs1)
    return np.round(sum3)

Then the values for the three inputs are calculated. Random numbers are used.

In [ ]:
in1_values = np.asarray(np.random.uniform(low=-5, high=5, size=(4,4)), dtype=np.float32)
in2_values = np.asarray(np.random.uniform(low=-5, high=5, size=(4,4)), dtype=np.float32)
in3_values = np.asarray(np.random.uniform(low=-5, high=5, size=(4,4)), dtype=np.float32)

We can easily pass the values to the function we just wrote to calculate the expected result. For the created model the inputs must be summarized in a dictionary, which is then passed on to the model.

In [ ]:
input_dict = {}
input_dict["in1"] = in1_values
input_dict["in2"] = in2_values
input_dict["in3"] = in3_values

To run the model and calculate the output, [onnxruntime](https://github.com/microsoft/onnxruntime) can be used. ONNX Runtime is a performance-focused complete scoring engine for ONNX models from Microsoft. The `.InferenceSession` function is used to create a session of the model and `.run` is used to execute the model.

In [ ]:
%pip install onnxruntime
import onnxruntime as rt

sess = rt.InferenceSession(onnx_model.SerializeToString())
output = sess.run(None, input_dict)

The input values are also transferred to the reference function. Now the output of the execution of the model can be compared with that of the reference.

In [ ]:
ref_output= expected_output(in1_values, in2_values, in3_values)
print("The output of the ONNX model is: \n{}".format(output[0]))
print("\nThe output of the reference function is: \n{}".format(ref_output))

if (output[0] == ref_output).all():
    print("\nThe results are the same!")
else:
    raise Exception("Something went wrong, the output of the model doesn't match the expected output!")

Now that we have verified that the model works as we expected it to, we can continue working with the graph.

### 1.2. How to manipulate an ONNX model

In the model there are two successive adder nodes. An adder node in ONNX can only add two inputs, but there is also the [**sum**](https://github.com/onnx/onnx/blob/master/docs/Operators.md#Sum) node, which can process more than two inputs. So it would be a reasonable change of the graph to combine the two successive adder nodes to one sum node.

In the following we assume that we do not know the appearance of the model, so we first try to identify whether there are two consecutive adders in the graph and then convert them into a sum node. 

Here we make use of QONNX compiler infrastructure. The QONNX repo provides a thin wrapper around the model which provides several additional helper functions to manipulate the graph. The code can be found [here](https://github.com/fastmachinelearning/qonnx/blob/main/src/qonnx/core/modelwrapper.py).

In [ ]:
import qonnx
from qonnx.core.modelwrapper import ModelWrapper
qonnx_model = ModelWrapper(onnx_model)

As explained in the previous section, it is important that the nodes are listed in the correct order. If a new node has to be inserted or an old node has to be replaced, it is important to do that in the appropriate position. The following function serves this purpose. It returns a dictionary, which contains the node name as key and the respective node index as value.

In [ ]:
def get_node_id(model):
    node_index = {}
    node_ind = 0
    for node in model.graph.node:
        node_index[node.name] = node_ind
        node_ind += 1
    return node_index

The function scans the list of nodes and stores a run index (`node_ind`) as node index in the dictionary for every node name.

Another helper function is being implemented that searches for adder nodes in the graph and returns the found nodes. This is needed to determine if and which adder nodes are in the given model.

In [ ]:
def identify_adder_nodes(model):
    add_nodes = []
    for node in model.graph.node:
        if node.op_type == "Add":
            add_nodes.append(node)
    return add_nodes

The function iterates over all nodes of the model and if the operation type is `"Add"` the node will be stored in `add_nodes`. At the end `add_nodes` is returned.

If we apply this to our model, three nodes should be returned.

In [ ]:
add_nodes = identify_adder_nodes(qonnx_model)
for node in add_nodes:
    print("Found adder node: {}".format(node.name))

Among other helper functions, `ModelWrapper` offers two functions that can help to determine the preceding and succeeding node of a node. However, these functions are not getting a node as input, but can determine the consumer or producer of a tensor. We write two functions that uses these helper functions to determine the previous and the next node of a node.

In [ ]:
def find_predecessor(model, node):
    predecessors = []
    for i in range(len(node.input)):
        producer = model.find_producer(node.input[i])
        predecessors.append(producer)
    return predecessors
        

def find_successor(model, node):
    successors = []
    for i in range(len(node.output)):
        consumer = model.find_consumer(node.output[i])
        successors.append(consumer)
    return successors

The first function uses `find_producer` from `ModelWrapper` to create a list of the producers of the inputs of the given node. So the returned list is indirectly filled with the predecessors of the node. The second function works in a similar way, `find_consumer` from `ModelWrapper` is used to find the consumers of the output tensors of the node and so a list with the successors can be created. 

In [ ]:
def adder_pair(model, node):
    adder_pairs = []
    node_pair = []
    successor_list = find_successor(model, node)
    
    for successor in successor_list:
        if successor.op_type == "Add":
            node_pair.append(node)
            node_pair.append(successor)
            adder_pairs.append((node_pair))
            node_pair = []
    return adder_pairs

The function gets a node and the model as input. Two empty lists are created to be filled with a list of adder node pairs that can be returned as result of the function. Then the function `find_successor` is used to return all of the successors of the node. If one of the successors is an adder node, the node is saved in `node_pair` together with the successive adder node and put in the list `adder_pairs`. Then the temporary list is cleaned and can be filled with the next adder node pair. Since it is theoretically possible for an adder node to have more than one subsequent adder node, a list of lists is created. This list of the node with all its successive adder nodes is returned.

So now we can find out which adder node has an adder node as successor. Since the model is known, one adder pair (Add1+Add2) should be found when applying the function to the previously determined adder node list (`add_nodes`).

In [ ]:
for node in add_nodes:
    add_pairs = adder_pair(qonnx_model, node)
    if len(add_pairs) != 0:
        for i in range(len(add_pairs)):
            substitute_pair = add_pairs[i]
            print("Found following pair that could be replaced by a sum node:")
            for node_pair in add_pairs:
                for node in node_pair:
                    print(node.name)

Now that the pair to be replaced has been identified (`substitute_pair`), a sum node can be instantiated and inserted into the graph at the correct position. 

First of all, the inputs must be determined. For this the adder nodes inputs are used minus the input, which corresponds to the output of the other adder node.

In [ ]:
input_list = []
for i in range(len(substitute_pair)):
    if i == 0:
        for j in range(len(substitute_pair[i].input)):
            if substitute_pair[i].input[j] != substitute_pair[i+1].output[0]:
                input_list.append(substitute_pair[i].input[j])
    else:
        for j in range(len(substitute_pair[i].input)):
            if substitute_pair[i].input[j] != substitute_pair[i-1].output[0]:
                input_list.append(substitute_pair[i].input[j])
print("The new node gets the following inputs: \n{}".format(input_list))

The output of the sum node matches the output of the second adder node and can therefore be taken over directly.

In [ ]:
sum_output = substitute_pair[1].output[0]

The summary node can be created with this information.

In [ ]:
Sum_node = onnx.helper.make_node(
    'Sum',
    inputs=input_list,
    outputs=[sum_output],
    name="Sum"
)

The node can now be inserted into the graph and the old nodes are removed.

In [ ]:
node_ids = get_node_id(qonnx_model)
node_ind = node_ids[substitute_pair[0].name]
graph.node.insert(node_ind, Sum_node)

for node in substitute_pair:
    graph.node.remove(node)

To insert the node in the right place, the index of the first node of the substitute_pair is used as node index for the sum node and embedded into the graph using `.insert`. Then the two elements in `substitute_pair` are deleted using `.remove`. `.insert` and `.remove` are functions provided by ONNX.

The new graph is saved as ONNX model and can be visualized with Netron.

In [ ]:
onnx_model1 = qonnx.util.basic.qonnx_make_model(graph, producer_name="simple-model1")
onnx.save(onnx_model1, 'simple_model1.onnx')

In [ ]:
showInNetron('simple_model1.onnx')

Through the visualization it can already be seen that the insertion was successful, but it is still to be checked whether the result remains the same. Therefore the result of the reference function written in the previous section is used and the new model with the input values is simulated. At this point onnxruntime can be used again. The simulation is analogous to the one of the first model in the previous section.

In [ ]:
sess = rt.InferenceSession(onnx_model1.SerializeToString())
output = sess.run(None, input_dict)

In [ ]:
print("The output of the manipulated ONNX model is: \n{}".format(output[0]))
print("\nThe output of the reference function is: \n{}".format(ref_output))

if (output[0] == ref_output).all():
    print("\nThe results are the same!")
else:
    raise Exception("Something went wrong, the output of the model doesn't match the expected output!")

# 2. QONNX - Analysis Passes

This part is about the analysis passes of QONNX.

We'll use the following utility functions to print the source code for function calls (`showSrc()`) and to visualize a network using netron (`showInNetron()`) in the Jupyter notebook:

In [ ]:
import inspect
import netron
import os
from IPython.display import IFrame
import urllib.request

def showSrc(what):
    print("".join(inspect.getsourcelines(what)[0]))

def download_model_from_zoo(qonnx_url, dl_file):
    urllib.request.urlretrieve(qonnx_url, dl_file)

### 2.1. Example - Quantity analysis of operation types
As an example, an analysis is designed that returns the number of nodes of the same operation types.

First the model is shown to illustrate the analysis. For this Netron is used. Netron is a visualizer for neural network, deep learning and machine learning models.

In [ ]:
download_model_from_zoo("https://github.com/fastmachinelearning/QONNX_model_zoo/raw/main/models/MNIST/Brevitas_FINN_TFC/TFC/TFC_2W2A.onnx", "TFC_2W2A.onnx")
showInNetron("TFC_2W2A.onnx")

Lets load the model with the `ModelWrapper` to work with it in QONNX.

In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
model = ModelWrapper('TFC_2W2A.onnx')

Lets start with what we did above and see if it also works on a more complex model

In [ ]:
add_nodes = identify_adder_nodes(model)
for node in add_nodes:
    print("Found adder node: {}".format(node.name))

The idea is to count all nodes that have the same operation type. The result should contain the operation types and the corresponding number of nodes that occur in the model. In the beginning an empty dictionary is created which is filled by the function and returned as result to the user at the end of the analysis.

In [ ]:
def count_unique_node_types(model):
    count_dict = {}
    for node in model.graph.node:
        if node.op_type in count_dict:
            count_dict[node.op_type] +=1
        else:
            count_dict[node.op_type] = 1
    return count_dict

The function takes the model as input and iterates over the nodes. Then it is checked whether there is already an entry for the operation type in the dictionary. If this is not the case, an entry is created and set to `1`. If there is already an entry, it is incremented. If all nodes in the model have been iterated, the filled dictionary is returned.

The analysis function of ModelWrapper is used to perform the analysis just designed. It is shown below and takes the function as input and performs it by passing the model to the function.

In [ ]:
showSrc(ModelWrapper.analysis)

The result can now simply be determined by calling the `.analysis` function.

In [ ]:
print(model.analysis(count_unique_node_types))

### 2.2. Calculate inference cost for QONNX model

QONNX has a command line tool to get the inference cost (qonnx-inference-cost). 

But we can also compute it in a python program. For this we will use the CNV_2W2A model.

In [ ]:
download_model_from_zoo("https://github.com/fastmachinelearning/qonnx_model_zoo/blob/main/models/CIFAR10/Brevitas_FINN_CNV/CNV_2W2A.onnx", "CNV_2W2A.onnx")
showInNetron("TFC_2W2A.onnx")

In [ ]:
from qonnx.util.inference_cost import inference_cost
model = ModelWrapper('TFC_2W2A.onnx')

In [ ]:
inference_cost(model)

# 3. QONNX - Transformation Passes

In this part the idea behind transformation passes in QONNX will be explained and with the help of an example the procedure of a transformation will be shown.


### .transform() from ModelWrapper

In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
showSrc(ModelWrapper.transform)

When the function is called, the model, the name of the transformation and, if required, the flag make_deepcopy are passed. It is also possible not to make a copy of the model. In this case `make_deepcopy` must be set to False. Then the branch `if make_deepcopy:` would not be taken and no copy of the model would be made. 

The unchanged model is first passed to the variable `transformed_model` to pass this variable on to the transformation later. 

`model_was_changed` indicates whether the transformation needs to be applied more then once. Because it needs to be applied at least one time `model_was_changed` is first set to True and then depending on the return values of the transformation function the transformation can be applied more then once. 

**Important**: Due to the structure of this function, `model_was_changed` must be set to False at some point. Otherwise the loop is infinite.
    

Each new transformation must correspond to the scheme of the base class and contain at least the function `apply(model)`, which returns the changed model and a bool value for `model_was_changed`.

In [ ]:
from qonnx.transformation.base import Transformation
showSrc(Transformation)

Base class is abstract class (`import ABC`) with only one abstract method (`apply()`) which gets the model as input. To show what a transformation should look like, the following example is taken from a general-purpose QONNX transformation.

## Example - ConvertSubToAdd
The transformation replaces all subtraction nodes in a model with addition nodes with appropriate sign. For that an onnx model is loaded which contains one subtraction node. 
    
Netron is used to visualize the result before and after.

In [ ]:
onnx_model = onnx.load('TFC_2W2A.onnx')
onnx_model = ModelWrapper(onnx_model)
showInNetron("TFC_2W2A.onnx")

In [ ]:
from qonnx.transformation.base import Transformation

class ConvertSubToAdd(Transformation):
    def apply(self, model):
        graph = model.graph
        for n in graph.node:
            if n.op_type == "Sub":
                A = model.get_initializer(n.input[1])
                if A is not None:
                    n.op_type = "Add"
                    model.set_initializer(n.input[1], -A)
        return (model, False)

First the transformation class must be imported. Then a class can be created for the new transformation, which is derived from the base class. In this case the transformation has only the `apply()` function. 

All nodes are checked by first extracting the graph from the model and then iterating over the node list. With the help of .op_type the operation type of the node can be checked, if the node is a subtraction node the condition `if n.op_type == "Sub"` is true. It may be that the subtraction input of the node has no value, this is checked with `model.get_initializer(n.input[1])`. 
    
    
**Important:** As with ONNX, QONNX always assumes a certain order of inputs, this is especially important if you want to create additional custom operation nodes.

When the input is initialized, the operation type of the node is converted to `"Add"`, this can simply be done by using the equal sign. Then the sign of the initial value must be changed. For this the ModelWrapper function `.set_initializer` can be used.

At the end the changed model is returned and `model_was_changed` is set to False, because the transformation has to be executed only once.



In [ ]:
onnx_model_transformed = onnx_model.transform(ConvertSubToAdd())
onnx_model_transformed.save('/tmp/LFCW1A1_changed.onnx')

In [ ]:
showInNetron('/tmp/LFCW1A1_changed.onnx')